# Data Acquisition using the LatissAcquireAndTakeSequence SAL script
The script is built for the scriptQueue but can be run via this notebook

This script is used to both acquire a target and put it within a radius of a pixel that is specific to the grating. The script can then be used to take a series of images with different filter and grating combinations. The focus and pointing adjustments associted with each filter and grating as performed automatically by the ATAOS (so long as the atspectrograph correction is enabled)

#### This example notebook is divided into three use-cases.
1. Perform a slew and acquisition only
2. Perform only the taking of data, without acquiring. This assumes the target is already positioned appropriately
3. Perform the acquisition and data taking as a single script

In [ ]:
import sys
import os
import asyncio
import time

import numpy as np
import logging 
import yaml
import astropy

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

## Instantiate the Script

First derive and index for the script that is specific to your user

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index=os.getuid()*10+np.random.randint(0,9)
logger.info(f'The generated index is {index}')

Instantiate the script then start all remotes

In [ ]:
script = LatissAcquireAndTakeSequence(index=index)
await script.start_task

Set the script to have a DEBUG log level

In [ ]:
script.log.level = logging.DEBUG

## Declare the Target

In [ ]:
target='HIP 31636'

Print the available instrument filter and gratings. <br>
This is useful both for slewing and for the configuration steps below

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

ATAOS must be on and corrections enabled, do as follows if required

In [ ]:
# await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

## Procedure 1 - Performing the acquisition ONLY

### Configure the script

The configurable parameters can be seen from either inside the script or from uncommenting the following command

In [ ]:
# script.get_schema()

Be sure to set the target, filter and grating to the appropriate setup you want to use for data taking. <br>
The position of the target on the detector is dictated by the choice of the grating.

In [ ]:
configuration = yaml.safe_dump({"object_name": target,
                                "do_acquire": True,
                                "acq_filter": 'BG40',
                                "acq_grating": 'ronchi170lpmm',
                                "acq_exposure_time": 2.0,
                                "target_pointing_tolerance": 3.0,       # Target positioning tolerance (arcsec)
                                "max_acq_iter": 3,                     # Maximum number of iterations to perform when acquiring
                                "target_pointing_verification": True,  # Takes an additional image showing the final target position
                               })
logger.info(configuration)

Set script state to UNCONFIGURED.
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

## Procedure 2 - Performing the science data collection ONLY

### Configure the script
The configurable parameters can be seen from either inside the script or from uncommenting the following command

In [ ]:
# script.get_schema()

Be sure to set the target, filter and grating to the appropriate setup you want to use for data taking. <br>
The position of the target on the detector is dictated by the choice of the grating.

In [ ]:
configuration = yaml.safe_dump({"object_name": target,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [30, 30, 30, 30,
                                                            30, 30, 30, 30,
                                                            30, 30, 30, 30,
                                                            30, 30, 30, 30,], 
                                "grating_sequence": ['ronchi170lpmm','ronchi170lpmm','ronchi170lpmm','ronchi170lpmm',
                                                    'ronchi170lpmm','ronchi170lpmm','ronchi170lpmm','ronchi170lpmm',
                                                    'holo4_003','holo4_003','holo4_003','holo4_003',
                                                    'holo4_003','holo4_003','holo4_003','holo4_003',],
                                "filter_sequence": ['empty_1','empty_1', 'BG40','BG40',
                                                    'quadnotch1','quadnotch1', 'FELH0600','FELH0600',
                                                    'empty_1','empty_1', 'BG40','BG40',
                                                    'quadnotch1','quadnotch1', 'FELH0600','FELH0600',], 
                                })
logger.info(configuration)

Set script state to UNCONFIGURED.
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

## Procedure 3 - Performing the acquisition AND the science data collection

### Configure the script
The configurable parameters can be seen from either inside the script or from uncommenting the following command

In [ ]:
# script.get_schema()

Be sure to set the target, filter and grating to the appropriate setup you want to use for data taking. <br>
The position of the target on the detector is dictated by the choice of the grating.

In [ ]:
configuration = yaml.safe_dump({"object_name": target,
                                "do_acquire": True,
                                "acq_filter": 'BG40',
                                "acq_grating": 'ronchi170lpmm',
                                "acq_exposure_time": 2.0,
                                "target_pointing_tolerance": 3.0,       # Target positioning tolerance (arcsec)
                                "max_acq_iter": 3,                     # Maximum number of iterations to perform when acquiring
                                "target_pointing_verification": True,  # Takes an additional image showing the final target position
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [30, 30, 30, 30,
                                                            30, 30, 30, 30,
                                                            30, 30, 30, 30,
                                                            30, 30, 30, 30,], 
                                "grating_sequence": ['ronchi170lpmm','ronchi170lpmm','ronchi170lpmm','ronchi170lpmm',
                                                    'ronchi170lpmm','ronchi170lpmm','ronchi170lpmm','ronchi170lpmm',
                                                    'holo4_003','holo4_003','holo4_003','holo4_003',
                                                    'holo4_003','holo4_003','holo4_003','holo4_003',],
                                "filter_sequence": ['empty_1','empty_1', 'BG40','BG40',
                                                    'quadnotch1','quadnotch1', 'FELH0600','FELH0600',
                                                    'empty_1','empty_1', 'BG40','BG40',
                                                    'quadnotch1','quadnotch1', 'FELH0600','FELH0600',],
                                })
logger.info(configuration)

Set script state to UNCONFIGURED.
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included. <br>
Therefore, it is included here despite being a non-operation in the first instance.

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

#### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
                groupId=astropy.time.Time.now().isot
            )
await script.do_setGroupId(group_id_data)
await script.arun()

If required, you can stop the telescope from tracking using the following cell, but remember you will lose your acquisition

In [ ]:
# await script.atcs.stop_tracking()